# 如何将运行时值传递给工具

```{=mdx}
:::info 预备知识

本指南假定您已熟悉以下概念：
- [聊天模型](/docs/concepts/chat_models)
- [LangChain 工具](/docs/concepts/tools)
- [如何创建工具](/docs/how_to/custom_tools)
- [如何使用模型调用工具](/docs/how_to/tool_calling/)
:::

:::info 支持的模型

本指南使用具有原生工具调用功能的模型。
您可以查看[支持工具调用的所有模型列表](/docs/integrations/chat/)。

:::
```

您可能需要将仅在运行时才知道的值绑定到某个工具。例如，工具的逻辑可能需要使用发起请求的用户ID。

大多数情况下，这些值不应由LLM控制。实际上，允许LLM控制用户ID可能会导致安全风险。

相反，LLM应该仅控制那些设计上应由LLM控制的工具参数，而其他参数（如用户ID）应由应用程序逻辑固定。

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs
  customVarName="llm"
/>
```

In [1]:
// @lc-docs-hide-cell

import { ChatOpenAI } from "@langchain/openai";

const llm = new ChatOpenAI({ model: "gpt-4o-mini" })

## 使用上下文变量

```{=mdx}
:::caution 兼容性
此功能是在 `@langchain/core>=0.3.10` 中添加的。如果您在项目中单独使用 LangSmith SDK，我们也建议升级到 `langsmith>=0.1.65`。请确保您的软件包已更新。

它还需要 [`async_hooks`](https://nodejs.org/api/async_hooks.html) 支持，在所有环境中并非都支持此功能。
:::
```

解决此问题的一种方法是使用**上下文变量**。上下文变量是一个强大的功能，允许您在应用程序的更高级别设置值，然后在从该级别调用的子 runnable（例如工具）中访问它们。

它们的工作方式超出了传统的范围规则，因此您无需直接引用已声明的变量即可访问其值。

下面，我们声明了一个工具，该工具根据名为 `userId` 的上下文变量更新中心 `userToPets` 状态。请注意，此 `userId` 不属于工具的 schema 或输入：

In [2]:
import { z } from "zod";
import { tool } from "@langchain/core/tools";
import { getContextVariable } from "@langchain/core/context";

let userToPets: Record<string, string[]> = {};

const updateFavoritePets = tool(async (input) => {
  const userId = getContextVariable("userId");
  if (userId === undefined) {
    throw new Error(`No "userId" found in current context. Remember to call "setContextVariable('userId', value)";`);
  }
  userToPets[userId] = input.pets;
  return "update_favorite_pets called."
}, {
  name: "update_favorite_pets",
  description: "add to the list of favorite pets.",
  schema: z.object({
    pets: z.array(z.string())
  }),
});

如果在更高层级设置上下文变量之前调用上述工具，`userId` 将会是 `undefined`：

In [3]:
await updateFavoritePets.invoke({ pets: ["cat", "dog" ]})

Error: No "userId" found in current context. Remember to call "setContextVariable('userId', value)";
    at updateFavoritePets.name (evalmachine.<anonymous>:14:15)
    at /Users/jacoblee/langchain/langchainjs/langchain-core/dist/tools/index.cjs:329:33
    at AsyncLocalStorage.run (node:async_hooks:346:14)
    at AsyncLocalStorageProvider.runWithConfig (/Users/jacoblee/langchain/langchainjs/langchain-core/dist/singletons/index.cjs:58:24)
    at /Users/jacoblee/langchain/langchainjs/langchain-core/dist/tools/index.cjs:325:68
    at new Promise (<anonymous>)
    at DynamicStructuredTool.func (/Users/jacoblee/langchain/langchainjs/langchain-core/dist/tools/index.cjs:321:20)
    at DynamicStructuredTool._call (/Users/jacoblee/langchain/langchainjs/langchain-core/dist/tools/index.cjs:283:21)
    at DynamicStructuredTool.call (/Users/jacoblee/langchain/langchainjs/langchain-core/dist/tools/index.cjs:111:33)
    at async evalmachine.<anonymous>:3:22


相反，请设置一个上下文变量，作为调用工具的位置的父级：

In [4]:
import { setContextVariable } from "@langchain/core/context";
import { BaseChatModel } from "@langchain/core/language_models/chat_models";
import { RunnableLambda } from "@langchain/core/runnables";

const handleRunTimeRequestRunnable = RunnableLambda.from(async (params: {
  userId: string;
  query: string;
  llm: BaseChatModel;
}) => {
  const { userId, query, llm } = params;
  if (!llm.bindTools) {
    throw new Error("Language model does not support tools.");
  }
  // Set a context variable accessible to any child runnables called within this one.
  // You can also set context variables at top level that act as globals.
  setContextVariable("userId", userId);
  const tools = [updateFavoritePets];
  const llmWithTools = llm.bindTools(tools);
  const modelResponse = await llmWithTools.invoke(query);
  // For simplicity, skip checking the tool call's name field and assume
  // that the model is calling the "updateFavoritePets" tool
  if (modelResponse.tool_calls.length > 0) {
    return updateFavoritePets.invoke(modelResponse.tool_calls[0]);
  } else {
    return "No tool invoked.";
  }
});

当我们的方法调用工具时，你将看到工具正确访问了先前设置的 `userId` 上下文变量，并成功运行：

In [5]:
await handleRunTimeRequestRunnable.invoke({
  userId: "brace",
  query: "my favorite animals are cats and parrots.",
  llm: llm
});

ToolMessage {
  "content": "update_favorite_pets called.",
  "name": "update_favorite_pets",
  "additional_kwargs": {},
  "response_metadata": {},
  "tool_call_id": "call_vsD2DbSpDquOtmFlOtbUME6h"
}


并且还更新了 `userToPets` 对象，添加了一个与我们传入的 `userId`（即 `"brace"`）匹配的键：

In [6]:
console.log(userToPets);

{ brace: [ 'cats', 'parrots' ] }


## 没有上下文变量的情况下

如果你使用的是较早版本的 core，或者所处的环境不支持 `async_hooks`，你可以使用以下设计模式，在运行时动态创建工具，并将其绑定到相应的值。

其思路是在请求时动态创建工具，并将其绑定到相应的信息。例如，
这些信息可以是从请求本身解析出的用户 ID。

In [7]:
import { z } from "zod";
import { tool } from "@langchain/core/tools";

userToPets = {};

function generateToolsForUser(userId: string) {
  const updateFavoritePets = tool(async (input) => {
    userToPets[userId] = input.pets;
    return "update_favorite_pets called."
  }, {
    name: "update_favorite_pets",
    description: "add to the list of favorite pets.",
    schema: z.object({
      pets: z.array(z.string())
    }),
  });
  // You can declare and return additional tools as well:
  return [updateFavoritePets];
}

验证工具是否正常工作

In [9]:
const [updatePets] = generateToolsForUser("cobb");

await updatePets.invoke({ pets: ["tiger", "wolf"] });

console.log(userToPets);

{ cobb: [ 'tiger', 'wolf' ] }


In [10]:
import { BaseChatModel } from "@langchain/core/language_models/chat_models";

async function handleRunTimeRequest(userId: string, query: string, llm: BaseChatModel): Promise<any> {
  if (!llm.bindTools) {
    throw new Error("Language model does not support tools.");
  }
  const tools = generateToolsForUser(userId);
  const llmWithTools = llm.bindTools(tools);
  return llmWithTools.invoke(query);
}

这段代码将允许LLM调用工具，但LLM**并不知道**存在**用户ID**这一事实。你可以看到，`user_id` 并不在LLM生成的参数列表中：

In [11]:
const aiMessage = await handleRunTimeRequest(
  "cobb", "my favorite pets are tigers and wolves.", llm,
);
console.log(aiMessage.tool_calls[0]);

{
  name: 'update_favorite_pets',
  args: { pets: [ 'tigers', 'wolves' ] },
  type: 'tool_call',
  id: 'call_FBF4D51SkVK2clsLOQHX6wTv'
}


```{=mdx}
:::提示
点击 [这里](https://smith.langchain.com/public/3d766ecc-8f28-400b-8636-632e6f1598c7/r) 查看上述运行的 LangSmith 跟踪。
:::

:::提示
聊天模型仅输出调用工具的请求。它们不会实际调用底层工具。

如需了解如何调用工具，请参阅 [如何使用模型调用工具](/docs/how_to/tool_calling/)。
:::
```